## Scrapper de Mercado Libre - Vehículos

El siguiente script tiene como finalidad la extraccion de informacion de automoviles publicados en Mercado Libre. La informacion se guardara en un archivo .csv y contendra los siguientes datos: marca, modelo, año de fabricación y kilometraje de los vehiculos.

In [ ]:
#Importamos las librerias necesarias

from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

In [ ]:
#Ingreso de las caracteristicas de busqueda deseadas

marca = input("Por favor, ingresa la marca del vehiculo: ")
modelo = input("Por favor, ingresa el modelo del vehiculo: ")
año_desde = input("Por favor, ingrese desde que año de fabricacion desea buscar el vehiculo. Si es indiferente indique 'desde': ")
año_hasta = input("Por favor, ingrese hasta que año de fabricacion desea buscar el vehiculo. Si es indiferente indique 'hasta': ")

if año_desde == "desde":
    notificacion_busqueda = f"Se procedera a buscar informacion sobre vehiculos marca {marca}, modelo {modelo}, fabricados hasta el año {año_hasta}"
elif año_hasta == "hasta":
    notificacion_busqueda = f"Se procedera a buscar informacion sobre vehiculos marca {marca}, modelo {modelo}, fabricados desde el año {año_desde}"
else:
    notificacion_busqueda = f"Se procedera a buscar informacion sobre vehiculos marca {marca}, modelo {modelo}, fabricados entre los años {año_desde} y {año_hasta}"

print(notificacion_busqueda)

In [ ]:
#Construccion de la dirección de URL

url_base = "https://autos.mercadolibre.com.ar"
def armar_url(url_base, marca, modelo, año_desde, año_hasta):
    mi_url = url_base
    mi_url+=f'/{marca}'
    mi_url+=f'/{modelo}'
    
    if año_desde == "desde":
        mi_url += f'/hasta-{año_hasta}'
    elif año_hasta == "hasta":
        mi_url += f'/desde-{año_desde}'
    else:
        mi_url += f'/{año_desde}-{año_hasta}'
    
    mi_url+=f'/{marca}-{modelo}_NoIndex_True#applied_filter_id%3DVEHICLE_YEAR%26applied_filter_name%3DA%C3%B1o%26applied_filter_order%3D7%26applied_value_id%3D'
    if año_hasta == "hasta":        
        mi_url+=f'*-{año_hasta}'
    elif año_desde == "desde":
        mi_url+=f'{año_desde}-*'
    else:
        mi_url += f'{año_desde}-{año_hasta}'
        
    mi_url += f'%26applied_value_name%3D'
    
    if año_hasta == "hasta":        
        mi_url+=f'*-{año_hasta}'
    elif año_desde == "desde":
        mi_url+=f'{año_desde}-*'
    else:
        mi_url += f'{año_desde}-{año_hasta}'
        
    mi_url += f'%26applied_value_order%3D10%26applied_value_results%3DUNKNOWN_RESULTS%26is_custom%3D'
            
    return mi_url

url = armar_url(url_base, marca, modelo, año_desde, año_hasta)
url

In [ ]:
#Carga y configuración del Driver

options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='C:/Users/Laut/Proyectos data/Latam_scraping/chromedriver', options=options)
info_autos = []

#Función para scrappear

def obtener_info(driver):
    driver.get(url)
    time.sleep(10)
    autos = driver.find_elements(By.XPATH,'//li[@class="ui-search-layout__item"]')

    for auto in autos:
     
        # Encuentra el modelo del auto dentro de la clase "ui-search-item__title shops__item-title"
        modelo = auto.find_element(By.CLASS_NAME, "ui-search-item__title").text.strip()

        # Encuentra los elementos li dentro de la clase "ui-search-card-attributes__attribute"
        attributes = auto.find_elements(By.CLASS_NAME, "ui-search-card-attributes__attribute")

        # Extrae el año de fabricación y el kilometraje (asumiendo que siempre están en ese orden)
        if len(attributes) >= 2:
            año_fabricacion = attributes[0].text.strip()
            kilometraje = attributes[1].text.strip()
        else:
            año_fabricacion = "No encontrado"
            kilometraje = "No encontrado"

        # Encuentra el elemento span con la clase "andes-money-amount__fraction" para el precio
        precio_element = auto.find_element(By.CLASS_NAME, "andes-money-amount__fraction")
        precio = precio_element.text.strip()

        # Encuentra el elemento span con la clase "andes-money-amount__currency-symbol" para la moneda
        moneda_element = auto.find_element(By.CLASS_NAME, "andes-money-amount__currency-symbol")
        moneda = moneda_element.text.strip()

        info_autos.append([modelo, año_fabricacion, kilometraje, precio, moneda])

    return info_autos

In [ ]:
#Ejecución del scrapper

current_page = 1
while True:
    print(f"Recopilando información de la página {current_page}...")
    obtener_info(driver)

    # Busqueda del botón "Siguiente" para ir a la siguiente página
    try:
        next_button = driver.find_element(By.CLASS_NAME, "andes-pagination__button--next")
        next_button_link = next_button.find_element(By.TAG_NAME, "a").get_attribute("href")
        if next_button_link:
            url = next_button_link
            current_page += 1
        else:
            break  # No hay más páginas
    except:
        break  # No se encontró el botón "Siguiente" o no hay más páginas

driver.close()
print("Ha finalizado la recopilación de informacion")

# Carga de la información en archivo CSV
csv_filename = "informacion_autos.csv"
with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Modelo", "Año de Fabricación", "Kilometraje", "Precio", "Moneda"])
    writer.writerows(info_autos)

print(f"La información se ha guardado en la lista 'info_autos' y exportada al archivo '{csv_filename}'")

In [ ]:
info_autos